This scripts are combine two scripts from 

- https://github.com/tensorflow/models/blob/master/slim/export_inference_graph.py

- https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph.py

you can use this two scripts directly with

```
python export_inference_graph.py --model_name=mobilenet_v1 \
    --default_image_size=224 \
    --output_file=mobilenet_v1.pb \


python freeze_graph.py --input_graph =mobilenet_v1.pb \
         --input_checkpoint=./checkpoints/mobilenet_v1.ckpt \
         --input_binary=true \
         --output_graph=mobilenet_v1.pb \
         --output_node_names=MobilenetV1/Predictions/Reshape_1
```

In [1]:
import tensorflow as tf

slim = tf.contrib.slim

In [2]:
import mobilenet_v1 as network

arg_scope = network.mobilenet_v1_arg_scope()
network_fn = network.mobilenet_v1

image_size = 224
num_classes = 1001
checkpoints = "./.checkpoints/mobilenet_v1/mobilenet_v1.ckpt"
output = "./.mobilenet_v1/mobilenet_v1.pb"
output_node_names = "MobilenetV1/Predictions/Reshape_1"

In [3]:
if not tf.train.checkpoint_exists(checkpoints):
    print("Input checkpoint '" + checkpoints + "' doesn't exist!")

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)
with tf.Graph().as_default() as graph:

    placeholder = tf.placeholder(name='input', dtype=tf.float32,
                                 shape=[1, image_size, image_size, 3])

    with slim.arg_scope(arg_scope):
        network_fn(placeholder, num_classes, is_training=False)
        
    with tf.Session() as sess:
        var_list = {}
        reader = tf.train.NewCheckpointReader(checkpoints)
        var_to_shape_map = reader.get_variable_to_shape_map()
        for key in var_to_shape_map:
            try:
                tensor = sess.graph.get_tensor_by_name(key + ":0")
            except KeyError:
                # This tensor doesn't exist in the graph (for example it's
                # 'global_step' or a similar housekeeping element) so skip it.
                continue
            var_list[key] = tensor
            
        saver = tf.train.Saver(var_list=var_list)
        saver.restore(sess, checkpoints)
        
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            graph.as_graph_def(),
            output_node_names.split(","),
            variable_names_blacklist=None)

        with tf.gfile.GFile(output, 'wb') as f:
            f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Restoring parameters from ./.checkpoints/mobilenet_v1/mobilenet_v1.ckpt
INFO:tensorflow:Froze 137 variables.
Converted 137 variables to const ops.
